In [ ]:
import os
from research_lib.utils.data_access_utils import S3AccessUtils, RDSAccessUtils
from weight_estimation.weight_estimator import WeightEstimator, CameraMetadata
from matplotlib import pyplot as plt

s3 = S3AccessUtils('/root/data')
RDS = RDSAccessUtils()

<h1> Rectification Library </h1>

In [ ]:
import json
from typing import Dict
import cv2
import numpy as np

IMAGE_WIDTH = 4096
IMAGE_HEIGHT = 3000


def get_camera_parameters(params: Dict) -> Dict:
    """Return individual camera parameters from JSON stereo parameters contents."""

    camera_matrix_1 = np.array(params['CameraParameters1']['IntrinsicMatrix']).transpose()
    camera_matrix_2 = np.array(params['CameraParameters2']['IntrinsicMatrix']).transpose()

    dist_coeffs_1 = params['CameraParameters1']['RadialDistortion'][0:2] + \
                    params['CameraParameters1']['TangentialDistortion'] + \
                    [params['CameraParameters1']['RadialDistortion'][2]]
    dist_coeffs_1 = np.array(dist_coeffs_1)

    dist_coeffs_2 = params['CameraParameters2']['RadialDistortion'][0:2] + \
                    params['CameraParameters2']['TangentialDistortion'] + \
                    [params['CameraParameters2']['RadialDistortion'][2]]
    dist_coeffs_2 = np.array(dist_coeffs_2)

    R = np.array(params['RotationOfCamera2']).transpose()
    T = np.array(params['TranslationOfCamera2']).transpose()

    image_size = (IMAGE_WIDTH, IMAGE_HEIGHT)
    (R1, R2, P1, P2, Q, leftROI, rightROI) = cv2.stereoRectify(camera_matrix_1, dist_coeffs_1,
                                                               camera_matrix_2,
                                                               dist_coeffs_2, image_size, R, T,
                                                               None,
                                                               None,
                                                               None, None, None,
                                                               cv2.CALIB_ZERO_DISPARITY, 0)
    left_maps = cv2.initUndistortRectifyMap(camera_matrix_1, dist_coeffs_1, R1, P1, image_size,
                                            cv2.CV_16SC2)
    right_maps = cv2.initUndistortRectifyMap(camera_matrix_2, dist_coeffs_2, R2, P2, image_size,
                                             cv2.CV_16SC2)

    params = {
        'left_maps': left_maps,
        'right_maps': right_maps,
        'camera_matrix_1': camera_matrix_1,
        'dist_coeffs_1': dist_coeffs_1,
        'R1': R1,
        'P1': P1,
        'camera_matrix_2': camera_matrix_2,
        'dist_coeffs_2': dist_coeffs_2,
        'R2': R2,
        'P2': P2
    }
    return params


def unrectify(ann: Dict, params: Dict):
    """Un-rectify ann with params."""

    left_maps = params['left_maps']
    right_maps = params['right_maps']

    ann_u = {'leftCrop': [], 'rightCrop': []}
    for side in ['leftCrop', 'rightCrop']:
        for item in ann[side]:
            bp = item['keypointType']
            x = item['xFrame']
            y = item['yFrame']
            if side == 'leftCrop':
                x_new, y_new = left_maps[0][y, x]
            elif side == 'rightCrop':
                x_new, y_new = right_maps[0][y, x]
            else:
                raise Exception('Invalid side!')

            ann_u[side].append({
                'keypointType': bp,
                'xFrame': x_new,
                'yFrame': y_new,
            })

    return ann_u


def rectify(ann: Dict, params: Dict) -> Dict:
    """Rectify ann with params."""

    camera_matrix_1 = params['camera_matrix_1']
    dist_coeffs_1 = params['dist_coeffs_1']
    R1 = params['R1']
    P1 = params['P1']

    camera_matrix_2 = params['camera_matrix_2']
    dist_coeffs_2 = params['dist_coeffs_2']
    R2 = params['R2']
    P2 = params['P2']

    ann_r = {'leftCrop': [], 'rightCrop': []}
    for side in ['leftCrop', 'rightCrop']:
        for item in ann[side]:
            bp = item['keypointType']
            x = item['xFrame']
            y = item['yFrame']
            if side == 'leftCrop':
                x_new, y_new = \
                    cv2.undistortPoints(
                        np.array([[x, y]]).astype(float),
                        camera_matrix_1,
                        dist_coeffs_1,
                        R=R1,
                        P=P1)[0][0]
            elif side == 'rightCrop':
                x_new, y_new = \
                    cv2.undistortPoints(
                        np.array([[x, y]]).astype(float),
                        camera_matrix_2,
                        dist_coeffs_2,
                        R=R2,
                        P=P2)[0][0]
            else:
                raise Exception('Invalid side!')

            ann_r[side].append({
                'keypointType': bp,
                'xFrame': x_new,
                'yFrame': y_new,
            })

    return ann_r


def un_re_rectify(df, stereo_params_o, stereo_params_n):
    params_o = get_camera_parameters(stereo_params_o)
    params_n = get_camera_parameters(stereo_params_n)

    ann_us, ann_u_rs = [], []
    for idx, row in df.iterrows():
        ann = row.annotation
        if ann is None:
            ann_u_rs.append(None)
            continue

        # un-rectify with original params
        ann_u = unrectify(ann, params_o)
        if ann_u:

            # re-rectify with new params
            ann_u_r = rectify(ann_u, params_n)
            ann_us.append(ann_u)
            ann_u_rs.append(ann_u_r)
        else:
            ann_us.append(None)
            ann_u_rs.append(None)

    df['ann_u'] = ann_us
    df['ann_u_r'] = ann_u_rs

In [ ]:
def get_camera_metadata(stereo_parameters):
    camera_metadata = {
        'focalLengthPixel': stereo_parameters['CameraParameters1']['FocalLength'][0],
        'baseline': abs(stereo_parameters['TranslationOfCamera2'][0] / 1e3),
        'focalLength': stereo_parameters['CameraParameters1']['FocalLength'][0] * 3.45e-6,
        'pixelCountWidth': 4096,
        'pixelCountHeight': 3000,
        'imageSensorWidth': 0.01412,
        'imageSensorHeight': 0.01035
    }
    
    return camera_metadata


def add_weights_u_r(df, camera_metadata):
    
    weight_model_f, _, _ = s3.download_from_url('https://aquabyte-models.s3-us-west-1.amazonaws.com/biomass/trained_models/2020-11-27T00-00-00/weight_model_synthetic_data.pb')
    kf_model_f, _, _ = s3.download_from_url('https://aquabyte-models.s3-us-west-1.amazonaws.com/k-factor/trained_models/2020-08-08T000000/kf_predictor_v2.pb')
    weight_estimator = WeightEstimator(weight_model_f, kf_model_f)

    pred_weights = []
    for idx, row in df.iterrows():
        ann = row.ann_u_r
        if ann is not None:
            cm = CameraMetadata(
                focal_length=camera_metadata['focalLength'],
                focal_length_pixel=camera_metadata['focalLengthPixel'],
                baseline_m=camera_metadata['baseline'],
                pixel_count_width=camera_metadata['pixelCountWidth'],
                pixel_count_height=camera_metadata['pixelCountHeight'],
                image_sensor_width=camera_metadata['imageSensorWidth'],
                image_sensor_height=camera_metadata['imageSensorHeight']
            )

            weight, _, _ = weight_estimator.predict(ann, cm)
            pred_weights.append(weight)
        else:
            pred_weights.append(None)
    
    df['weight_u_r'] = pred_weights

<h1> Load dataset </h1>

In [ ]:
def extract_biomass_data(pen_id, start_date, end_date, akpd_score_cutoff):
    """Get raw biomass computations for given pen_id, date range, and AKPD score cutoff."""

    query = """
        SELECT * FROM
        prod.biomass_computations bc
        WHERE bc.pen_id={}
        AND bc.akpd_score >= {}
        AND bc.captured_at BETWEEN '{}' and '{}'
        AND bc.estimated_weight_g > 0.0
    """.format(pen_id, akpd_score_cutoff, start_date, end_date)

    df = RDS.extract_from_database(query)
    return df

In [ ]:
pen_id = 145
start_date = '2021-02-01'
end_date = '2021-02-04'
akpd_score_cutoff = 0.95
df = extract_biomass_data(pen_id, start_date, end_date, akpd_score_cutoff)

<h1> Un- and Re-rectify key-points </h1>

In [ ]:
stereo_parameters_o_s3_url = 's3://aquabyte-stereo-parameters/L40029797_R40020184/2020-10-05T22:39:45.664664000Z_L40029797_R40020184_stereo-parameters.json'
stereo_parameters_o_components = stereo_parameters_o_s3_url.replace('s3://', '').split('/')
bucket = stereo_parameters_o_components[0]
key = os.path.join(*stereo_parameters_o_components[1:])
stereo_parameters_o_f = s3.download_from_s3(bucket, key)

stereo_parameters_n_f = '/root/data/alok/biomass_estimation/playground/stereo_params_dale_p3_before_assets_10_5_2020.json'
stereo_params_o = json.load(open(stereo_parameters_o_f))
stereo_params_n = json.load(open(stereo_parameters_n_f))
un_re_rectify(df, stereo_params_o, stereo_params_n)
cm_tangential_fix = get_camera_metadata(stereo_params_n)
add_weights_u_r(df, cm_tangential_fix)



<h1> Verify that un-rectified annotations are correct </h1>

In [ ]:
def display_crops(left_image_f, right_image_f, ann, overlay_keypoints=True, show_labels=True):

    fig, axes = plt.subplots(2, 1, figsize=(20, 20))
    left_image = plt.imread(left_image_f)
    right_image = plt.imread(right_image_f)
    axes[0].imshow(left_image)
    axes[1].imshow(right_image)
    
    left_keypoints = {item['keypointType']: [item['xCrop'], item['yCrop']] for item in ann['leftCrop']}
    right_keypoints = {item['keypointType']: [item['xCrop'], item['yCrop']] for item in ann['rightCrop']}
    
    if overlay_keypoints:
        for bp, kp in left_keypoints.items():
            axes[0].scatter([kp[0]], [kp[1]], color='red', s=1)
            if show_labels:
                axes[0].annotate(bp, (kp[0], kp[1]), color='red')
        for bp, kp in right_keypoints.items():
            axes[1].scatter([kp[0]], [kp[1]], color='red', s=1)
            if show_labels:
                axes[1].annotate(bp, (kp[0], kp[1]), color='red')
    plt.show()

In [ ]:
for idx, row in df.iterrows():
    ann = row.annotation
    left_crop_url = row.left_crop_url
    right_crop_url = row.right_crop_url
    left_crop_f, _, _ = s3.download_from_url(left_crop_url)
    right_crop_f, _, _ = s3.download_from_url(right_crop_url)
    display_crops(left_crop_f, right_crop_f, ann)
    
    
    
    

In [ ]:
for idx, row in df.iterrows():
    ann = row.ann_u
    left_crop_url = row.left_crop_url.replace('aquabyte-crops', 'aquabyte-frames-resized-inbound')
    right_crop_url = row.right_crop_url.replace('aquabyte-crops', 'aquabyte-frames-resized-inbound')
    left_crop_f, _, _ = s3.download_from_url(left_crop_url)
    right_crop_f, _, _ = s3.download_from_url(right_crop_url)
    left_bbox = [int(x) for x in os.path.basename(left_crop_f).replace('.jpg', '').split('_')[-4:]]
    right_bbox = [int(x) for x in os.path.basename(right_crop_f).replace('.jpg', '').split('_')[-4:]]
    
    new_ann = {'leftCrop': [], 'rightCrop': []}
    for item in ann['leftCrop']:
        new_item = dict(item)
        new_item['xCrop'] = item['xFrame'] - left_bbox[0]
        new_item['yCrop'] = item['yFrame'] - left_bbox[1]
        new_ann['leftCrop'].append(new_item)
    for item in ann['rightCrop']:
        new_item = dict(item)
        new_item['xCrop'] = item['xFrame'] - right_bbox[0]
        new_item['yCrop'] = item['yFrame'] - right_bbox[1]
        new_ann['rightCrop'].append(new_item)
        
    display_crops(left_crop_f, right_crop_f, new_ann)
    
    
    
    

In [ ]:
left_bbox = [int(x) for x in os.path.basename(left_crop_f).replace('.jpg', '').split('_')[-4:]]

In [ ]:
left_bbox

In [ ]:
row.left_crop_metadata

In [ ]:
df.estimated_weight_g.mean()

In [ ]:
df.weight_u_r.mean()

In [ ]:
plt.figure(figsize=(10, 5))
plt.scatter(df.estimated_weight_g, df.weight_u_r)
plt.grid()
plt.xlabel('Dale P3 predicted weight - tangential distortion unaccounted for')
plt.ylabel('Dale P3 predicted weight - tangential distortion accounted for')
plt.show()

In [ ]:
plt.hist(((df.weight_u_r - df.estimated_weight_g)/df.estimated_weight_g).values)